In [0]:
%tensorflow_version 1.x # 確保 colob 中使用的 tensorflow 是 1.x 版本而不是 tensorflow 2
import tensorflow as tf

print(tf.__version__)

In [0]:
# 連接Google雲端硬碟，切換到工作目錄
from google.colab import drive 

drive.mount('/content/gdrive')
%cd 'gdrive/My Drive/Day49_50'
!ls

In [0]:
# 檢查 yolov3 的網路權重 yolo.h5
import os

if not os.path.exists("model_data/yolo.h5"):
  # 下載 yolov3 的網路權重，並且把權重轉換為 keras 能夠讀取的格式
  print("Model doesn't exist, downloading...")
  os.system("wget https://pjreddie.com/media/files/yolov3.weights")
  print("Converting yolov3.weights to yolo.h5...")
  os.system("python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5")
else:
  print("Model exist")

In [0]:
# 經由資料集產生train.txt 及 val.txt

if not os.path.exists("train.txt"): # 範例中訓練模型時所使用的，已經做好轉換的 annotation 檔名，增加這個檢查避免每次重新跑這段轉換的程式碼
  import xml.etree.ElementTree as ET # 載入能夠 Parser xml 文件的 library
  #from os import getcwd

  sets = ['train', 'val']

  # 資料類別
  classes = ["kangaroo", "raccoon"]
             
  # 把 annotation 轉換訓練時需要的資料形態
  def convert_annotation(image_id, list_file):
      in_file = open('data/Annotations/%s.xml'%(image_id))
      tree = ET.parse(in_file)
      root = tree.getroot()

      for obj in root.iter('object'):
          difficult = obj.find('difficult').text
          cls = obj.find('name').text
          if cls not in classes or int(difficult)==1:
              continue
          cls_id = classes.index(cls)
          xmlbox = obj.find('bndbox')
          b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
          list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))

  wd = "."
  count = 75  # 受限於硬體資源每個類別可處理的張數 20, 30, 50, 60, 75
  for image_set in sets:
      image_ids = open('data/ImageSets/%s%s.txt'%(image_set, count)).read().strip().split()
      annotation_path = '%s.txt'%(image_set)
      list_file = open(annotation_path, 'w')
      print("save annotation at %s" % annotation_path)
      for image_id in image_ids[:count * 2]: # 只處理 count * 2 張圖片來做範例
          print(wd, image_id)
          list_file.write('%s/data/TrainImages/%s.jpg'%(wd, image_id))
          convert_annotation(image_id, list_file)
          list_file.write('\n')
      list_file.close()
else:
  print('file exist.')

### Begin Training

In [0]:
# 將 train.py 所需要的套件載入
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

from train import get_classes, get_anchors, create_model, create_tiny_model, data_generator, data_generator_wrapper

In [0]:
#把 YOLO weights 轉換為能夠提供給 keras 作為訓練新模型的初始權重
import os

if not os.path.exists("model_data/yolo_weights.h5"):
  print("Converting pretrained YOLOv3 weights for training")
  os.system("python convert.py -w yolov3.cfg yolov3.weights model_data/yolo_weights.h5") 
else:
  print("Pretrained weights exists")

In [0]:
# 開始進行 Training 及 Validation

annotation_path = 'train.txt' # 轉換好格式的標註檔案
log_dir = 'logs/000/' # 訓練好的模型儲存的路徑
classes_path = 'model_data/kr_classes.txt'
anchors_path = 'model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (416,416) # multiple of 32, hw

is_tiny_version = len(anchors)==6 # default setting
if is_tiny_version:
    model = create_tiny_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='model_data/tiny_yolo_weights.h5')
else:
    model = create_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='model_data/yolo_weights.h5') # make sure you know what you freeze

logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

# 分為 training 以及 validation
val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

# Train with frozen layers first, to get a stable loss.
# Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
# 一開始先 freeze YOLO 除了 output layer 以外的 darknet53 backbone 來 train
if True:
    model.compile(optimizer=Adam(lr=1e-3), loss={
        # use custom yolo_loss Lambda layer.
        'yolo_loss': lambda y_true, y_pred: y_pred})

    batch_size = 16
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    # 模型利用 generator 產生的資料做訓練，強烈建議大家去閱讀及理解 data_generator_wrapper 在 train.py 中的實現
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=50,
            initial_epoch=0,
            callbacks=[logging, checkpoint])
    model.save_weights(log_dir + 'trained_weights_stage_1.h5')

# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.
if True:
    # 把所有 layer 都改為 trainable
    for i in range(len(model.layers)):
        model.layers[i].trainable = True
    model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
    print('Unfreeze all of the layers.')

    batch_size = 16 # note that more GPU memory is required after unfreezing the body
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size),
        epochs=100,
        initial_epoch=50,
        callbacks=[logging, checkpoint, reduce_lr, early_stopping])
    model.save_weights(log_dir + 'trained_weights_final.h5')

### Image Test

In [0]:
# 載入測試的圖片
from PIL import Image

image = Image.open('data/TestImages/r001.jpg') 

In [0]:
#進行圖片測試
from yolo import YOLO

yolo_model = YOLO(model_path='logs/000/trained_weights_final.h5', classes_path="model_data/kr_classes.txt")
r_image = yolo_model.detect_image(image, True)

In [0]:
#顯示測試後的圖片
r_image

### Video Test

In [0]:
# 方法一： 透過 https://github.com/erhwenkuo/deep-learning-with-keras-notebooks/blob/master/3.5-yolov2-train-hands-dataset.ipynb 進行影片測試

import cv2
import numpy as np
from yolo import YOLO
from PIL import Image

log_dir = 'logs/000/'
yolo_model = YOLO(model_path=log_dir + 'trained_weights_final.h5', classes_path="model_data/kr_classes.txt")

# 偵測的輸入影像檔
video_inp =  "data/Vedios/Kangaroo.mp4"

# 偵測結果的輸出影像檔
video_out =  "data/Vedios/Kangaroo-out.mp4"

# 透過OpenCv擷取影像
video_reader = cv2.VideoCapture(video_inp)

# 取得影像的基本資訊
nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT)) # 總共有多少frames
nb_FPS = int(video_reader.get(cv2.CAP_PROP_FPS)) # FPS
frame_h = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))  # 每個frame的高
frame_w = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))   # 每個frame的寬

# 設定影像的輸出
video_writer = cv2.VideoWriter(video_out, cv2.VideoWriter_fourcc(*'XVID'), nb_FPS, (frame_w, frame_h))

# 迭代每一個frame來進行圖像偵測
for i in range(nb_frames - 1):
    if i % 10 == 0: print(i, '/', nb_frames)
    # 讀取一個frame
    ret, cvimage = video_reader.read()

    # CV2 Image 轉換成PIL Image, 並進行圖像偵測
    pilimage = Image.fromarray(np.uint8(cvimage[:,:,::-1]))
    p_image = yolo_model.detect_image(pilimage, False)

    # PIL Image 轉換成CV2 Image, 並將圖片解入 video
    c_image = np.uint8(np.asarray(p_image)[:,:,::-1])
    video_writer.write(c_image)

video_reader.release() # 釋放資源
video_writer.release() # 釋放資源

In [0]:
# 方法二：透過yolo提供的方法detect_video進行影片測試

import yolo
from yolo import YOLO

log_dir = 'logs/000/'
yolo_model = YOLO(model_path=log_dir + 'trained_weights_final.h5', classes_path="model_data/kr_classes.txt")

# 偵測的輸入影像檔
video_inp =  "data/Vedios/Raccoon.mp4"
# 偵測結果的輸出影像檔
video_out =  "data/Vedios/Raccoon-1.mp4"

yolo.detect_video(yolo_model, video_inp, video_out)